In [14]:
from typing import List, Dict
import nixio
from os.path import join
from os import PathLike, listdir
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import regex as re

In [29]:
def main():
    for nixdir in nixdirs():
        nixfiles = _find_nix_files(nixdir)
        for subject, sessions in nixfiles.items():
            for session, path in sessions.items():
                print(subject, session, "->", path)
                nixfile = nixio.File.open(path, nixio.FileMode.ReadOnly)
                df = _read_session(nixfile, subject, session)
                print(df.to_string())

In [16]:
def nixdirs() -> List[PathLike]:
    return [_compose_nixdir(subpath) for subpath in _dataset_subpaths()]

In [17]:
def _dataset_subpaths() -> List[PathLike]:
    return [
        join("Human_Amygdala_MUA_sEEG_FearVideo", "data_NIX"),
        join("Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM", "data_NIX"),
        "Human_MTL_units_visual_WM"
    ]

In [18]:
def _compose_nixdir(subpath: PathLike) -> PathLike:
    base_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH")
    return join(base_dir, subpath)

In [19]:
def _find_nix_files(nixdir: PathLike) -> Dict[int, Dict[int, PathLike]]:
    nix_files = {}
    for file in listdir(nixdir):
        match = _NIX_REGEX.match(file)
        if match:
            subject, session = map(int, match.groups())
            if subject not in nix_files:
                nix_files[subject] = {}
            nix_files[subject][session] = join(nixdir, file)
    return nix_files

In [20]:
_NIX_REGEX = re.compile(r"Data_Subject_(\d{2})_Session_(\d{2}).h5")

In [21]:
def _read_session(nixfile: nixio.File, subject: int, session: int) -> pd.DataFrame:
    subject_dict = _read_subject(nixfile)
    trials = _read_trials(nixfile)

    subject_df = pd.DataFrame([subject_dict] * len(trials))
    df = pd.concat([subject_df, trials], axis=1)
    df["subject"] = subject
    df["session"] = session
    df.reset_index(drop=True, inplace=True)
    return df

In [22]:
def _read_subject(nixfile: nixio.File) -> Dict[str, str]:
    subject = nixfile.sections["Subject"]
    return _read_into_dict(subject.props)

In [23]:
def _read_trials(nixfile: nixio.File) -> pd.DataFrame:
    trials = nixfile.sections["Session"].sections["Trial properties"]
    trial_dicts = [_read_trial(trial) for trial in trials]
    return pd.DataFrame(trial_dicts)


In [24]:
def _read_trial(trial: nixio.Section) -> Dict[str, str]:
    return _read_into_dict(trial.props)


In [25]:
def _read_into_dict(iterable) -> Dict[str, str]:
    props_dict = {}
    for item in iterable:
        name = item.name
        value = item.values[0]
        props_dict[name] = value

    return props_dict

In [30]:
if __name__ == "__main__":
    main()

1 1 -> /mnt/c/Users/conta/git/USZ_NCH/Human_Amygdala_MUA_sEEG_FearVideo/data_NIX/Data_Subject_01_Session_01.h5
     Age Gender  Pathology Depth electrodes Electrodes in seizure onset zone (SOZ)  Trial number  Condition Condition name  subject  session
0   31.0      M  sclerosis           AL, AR                                     AR           1.0        1.0        Neutral        1        1
1   31.0      M  sclerosis           AL, AR                                     AR           2.0        2.0       Aversive        1        1
2   31.0      M  sclerosis           AL, AR                                     AR           3.0        1.0        Neutral        1        1
3   31.0      M  sclerosis           AL, AR                                     AR           4.0        2.0       Aversive        1        1
4   31.0      M  sclerosis           AL, AR                                     AR           5.0        1.0        Neutral        1        1
5   31.0      M  sclerosis           AL, AR

KeyboardInterrupt: 